## Entity–relationship model

In [ ]:
CREATE OR REPLACE TABLE enrollments (
    -- Create a new entity with a unique identifier 
    enrollment_id NUMBER(10,0) PRIMARY KEY
    -- Add the entity's attributes
    year NUMBER(4,0),    
    semester VARCHAR(255) 
    -- Add relationships to other entities
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (class_id) REFERENCES classes(class_id)
);

In [ ]:
# Retrieving data from the ER Model 
SELECT students.name,    
classes.class_name 
FROM students 
JOIN enrollments ON students.student_id = enrollments.student_id 
JOIN classes ON enrollments.class_id = classes.class_id 
JOIN schools ON classes.department_id = schools.school_id 
WHERE enrollments.year ='2023' 
AND schools.school_name ='Science';

### Creating entities for ER model
The HR department has requested a new system to track the training programs employees have completed.

Your task involves applying the ER model by setting up a new entity to record the details of each session, including a reference to the training program from the trainings entity.

In [ ]:
-- Create new entity
CREATE OR REPLACE TABLE employee_training_details (
  	-- Assign a unique identifier for the entity
	employee_training_id NUMBER(10,0) PRIMARY KEY,
  	-- Add new attribute
    year NUMBER(4,0),
  	-- Add new attributes to reference foreign entities
  	employee_id NUMBER(38,0),
    training_id NUMBER(38,0),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
    FOREIGN KEY (training_id) REFERENCES trainings(training_id)
);

### Retrieving data from ER model
The HR department has launched an initiative to closely monitor employees training progress within the 'Operations' department. They aim to identify employees who have exceeded the average training score threshold, indicating successful completion of their training programs.

Your task is to use the ER model to build a report by joining relevant entities and applying necessary filters to get the results fast and efficiently.

In [ ]:
SELECT 
	-- Add attributes to select
	employees.employee_id, 
    trainings.avg_training_score
FROM employees
	-- Merge entities on common keys
	JOIN trainings
	ON employees.employee_id = trainings.employee_id
LIMIT 50;

In [ ]:
SELECT 
	employees.employee_id, 
    trainings.avg_training_score
FROM employees
	JOIN trainings 
	ON employees.employee_id = trainings.employee_id
-- Add filter
WHERE trainings.avg_training_score > 65
LIMIT 50;

In [ ]:
SELECT 
	employees.employee_id, 
    trainings.avg_training_score
FROM employees
	JOIN trainings 
	ON employees.employee_id = trainings.employee_id
    -- Merge new entity
    JOIN departments 
    ON employees.department_id = departments.department_id
WHERE trainings.avg_training_score > 65
	-- Add extra filter
	AND departments.department_name = 'Operations'
LIMIT 50;

## Dimensional Modeling

In [ ]:
# Defining dimensions 
-- Rename entities to dim_EntityName for clarity, following up dimensions in the model: 
ALTER TABLE students RENAME TO dim_students;
ALTER TABLE classes RENAME TO dim_classes;
ALTER TABLE schools RENAME TO dim_schools;

In [ ]:
# Defining date dimension 
-- Creating the dim_date table to store key fixed dates related to student enrollments inschool:
CREATE OR REPLACE TABLE dim_date (  
    date_id NUMBER(10,0) PRIMARY KEY,
year NUMBER(4,0), 
semester VARCHAR(255)
);

In [ ]:
# Defining enrollments fact 
-- Create a fact entity containing references to all the dimensions:
CREATE OR REPLACE TABLE fact_enrollments ( 
    enrollment_id NUMBER(10,0) PRIMARY KEY,  
student_id NUMBER(10,0),    
class_id NUMBER(10,0),   
date_id NUMBER(10,0),
FOREIGN KEY (student_id) REFERENCES dim_students(student_id),
FOREIGN KEY (class_id) REFERENCES dim_classes(class_id),
FOREIGN KEY (date_id) REFERENCES dim_date(date_id)
);

In [ ]:
# Retrieving data from the dimensions 
SELECT name,   
class_name 
FROM fact_enrollments 
JOIN dim_students -- Joining to get student names 
ON fact_enrollments.student_id = dim_students.student_id 
JOIN dim_classes -- Joining to get class names 
ON fact_enrollments.class_id = dim_classes.class_id
JOIN dim_schools -- Joining to filter for the 'Science' school 
ON dim_classes.school_id = dim_schools.school_id
JOIN dim_date -- Joining to restrict data to the year 2023
ON fact_enrollments.date_id = dim_date.date_id
WHERE dim_schools.school_name ='Science' 
AND dim_date.year = 2023;

### Preparing dimensions
The HR department has data from employees and what training they have been doing. They have asked you to provide a model to quickly analyze large data sets, efficiently integrate reports and their business intelligence tools, and help them with data-driven decision-making.

Your task is to prepare the dimensions to connect to the fact table in the center of the star schema dimensions model.

In [ ]:
-- Modify entity
ALTER TABLE IF EXISTS employees
RENAME TO dim_employees;

ALTER TABLE IF EXISTS departments 
RENAME TO dim_departments;

ALTER TABLE IF EXISTS trainings
RENAME TO dim_trainings;

### Creating dimensions
The company requires employees to take training yearly to stay up to date. As part of creating a report for HR employees and their training, it is necessary also to register the last date when the employee took the training. Your task is to introduce a new dimension to the model to document this event. Once you do this, it will finalize the fact table, linking together all the dimensions:

In [ ]:
-- Create new entity
CREATE OR REPLACE TABLE dim_date (
  	-- Add unique identifier
    date_id NUMBER(10,0) PRIMARY KEY,
  	-- Add new attributes to register date
    year NUMBER(4,0),
    month NUMBER(2,0)
);

### Retrieving data from dimensional model
With the new dimensional model in place, the HR department wants to analyze the training sessions attended by all the employees in 2023. They are trying to gather the employees' average score, which is lower than the minimum 100 points, to call them for extra training.

Your task is to extract this information from the data warehouse.

In [ ]:
SELECT 
	-- Retrieve all attributes from the dimension
	dim_employees.*
FROM fact_employee_trainings 
	-- Merge fact table with dimension
	JOIN dim_employees
    ON fact_employee_trainings.employee_id = dim_employees.employee_id;

In [ ]:
SELECT 
	dim_employees.*,
    -- Retrieve average training scores
    dim_trainings.avg_training_score
FROM fact_employee_trainings 
	JOIN dim_employees
    ON fact_employee_trainings.employee_id = dim_employees.employee_id
    -- Merge fact table with dimension
    JOIN dim_trainings
    ON fact_employee_trainings.training_id = dim_trainings.training_id
-- Add filter
WHERE dim_trainings.avg_training_score < 100;

In [ ]:
SELECT 
	dim_employees.*,
    dim_trainings.avg_training_score,
    -- Add new attribute
    dim_departments.department_name
FROM fact_employee_trainings 
	JOIN dim_employees
    ON fact_employee_trainings.employee_id = dim_employees.employee_id
    JOIN dim_trainings 
    ON fact_employee_trainings.training_id = dim_trainings.training_id
    -- Add dimension needed
	JOIN dim_departments 
    ON fact_employee_trainings.department_id = dim_departments.department_id
WHERE dim_trainings.avg_training_score < 100;

In [ ]:
SELECT 
	dim_employees.*,
    dim_trainings.avg_training_score,
    dim_departments.department_name
FROM fact_employee_trainings 
	JOIN dim_employees
    ON fact_employee_trainings.employee_id = dim_employees.employee_id
    JOIN dim_trainings 
    ON fact_employee_trainings.training_id = dim_trainings.training_id
    JOIN dim_departments 
    ON fact_employee_trainings.department_id = dim_departments.department_id
    -- Add dimension needed
    JOIN dim_date  
    ON fact_employee_trainings.date_id = dim_date.date_id
WHERE dim_trainings.avg_training_score < 100   
    -- Add extra filter
    AND dim_date.year = 2023;

## Data Vault

In [ ]:
# Building hubs (1) 
-- AUTOINCREMENT: Attribute property to automatically generate unique, sequentially increasing numeric values for each new row.
CREATE OR REPLACE TABLE hub_students (  
    student_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY
);

# Building hubs (2) 
-- Create a new hub with a unique numerical key generated automatically and the hub's concept id:
CREATE OR REPLACE TABLE hub_students (   
    student_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY, 
    student_id NUMBER(10,0)
);

# Building hubs (3) 
-- Add historical tracking attributes:
CREATE OR REPLACE TABLE hub_students (  
    student_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY, 
    student_id NUMBER(10,0), 
    load_date TIMESTAMP,  
    record_source VARCHAR(255)
);

In [ ]:
# Building links (1) 
-- Create a link entity with a unique numerical key generated automatically: 
CREATE OR REPLACE TABLE link_enrollments ( 
    link_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY
);

# Building links (2) 
-- Add relationships to other entities: 
CREATE OR REPLACE TABLE link_enrollments (  
    link_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,  
    student_key NUMBER(10,0),  
    class_key NUMBER(10,0),
    FOREIGN KEY (student_key) REFERENCES hub_students(student_key),
    FOREIGN KEY (class_key) REFERENCES hub_classes(class_key)
    );

### Creating hubs
The HR department has commented that they need a more detailed history of all the changes with their records of employees, departments, and training.

You will need to restructure its data into a series of hubs.

Hubs will create a scalable and agile data warehouse that can accommodate changes over time without disrupting existing structures.

In [ ]:
-- Create a new hub entity
CREATE OR REPLACE TABLE hub_employee (
	-- Assign automated values to the hub key
	hub_employee_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
	employee_id NUMBER(38,0),
	-- Add attributes for historical tracking
	load_date TIMESTAMP,
	record_source VARCHAR(255)
);

In [ ]:
CREATE OR REPLACE TABLE hub_department (
	-- Assign automated values to the hub key
	hub_department_id NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
  	-- Add hubs key reference
	department_id NUMBER(38,0),
	-- Add attributes for historical tracking
	load_date TIMESTAMP,
	record_source VARCHAR(255)
);

In [ ]:
CREATE OR REPLACE TABLE hub_training (
	-- Add hub key
	hub_training_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
    -- Add the key attribute of trainings
	training_id NUMBER(38,0),
	-- Add history tracking attributes
	load_date TIMESTAMP,
	record_source VARCHAR(255)
);

### Creating satellites
With the hubs established, the next step is to create satellites that store descriptive information related to each hub. Satellites provide context and depth to the keys stored in hubs, and they capture the descriptive attributes, changes over time, and the history of business concepts.

Your task will be to create these satellites.

In [ ]:
-- Create a new satellite
CREATE OR REPLACE TABLE sat_employee (
	sat_employee_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
	hub_employee_key NUMBER(10,0) UNIQUE,
   	employee_name VARCHAR(255),
    gender CHAR(1),
    age NUMBER(3,0),
	-- Add history tracking attributes
	load_date TIMESTAMP,
    record_source VARCHAR(255),
	-- Add a reference to foreign hub
    FOREIGN KEY (hub_employee_key) REFERENCES hub_employee(hub_employee_key)
);

In [ ]:
CREATE OR REPLACE TABLE sat_department (
	-- Add the satellites unique identifier
	sat_department_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
	-- Add the hubs key attribute
	hub_department_key NUMBER(10,0) UNIQUE,
	department_name VARCHAR(255),
	region VARCHAR(255),
    -- Add history tracking attributes
    load_date TIMESTAMP,
    record_source VARCHAR(255),
	-- Add a reference to foreign hub
	FOREIGN KEY (hub_department_key) REFERENCES hub_department(hub_department_key)
);

In [ ]:
CREATE OR REPLACE TABLE sat_training (
	sat_training_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
	-- Add the hub's key reference
	hub_training_key NUMBER(10,0) UNIQUE,
	training_type VARCHAR(255),
    duration NUMBER(4,0),
    trainer_name VARCHAR(255),
    load_date TIMESTAMP,
    record_source VARCHAR(255),
	-- Add a reference to foreign hub
	FOREIGN KEY (hub_training_key) REFERENCES hub_training(hub_training_key) 
);

### Creating links
The final piece in the data vault model for HR is the creation of Links. Links represent relationships between hubs or capture transactions and interactions involving multiple hubs.

Your task is creating a link entity representing the relationship or interaction between the employee, training, and department hubs.

In [ ]:
-- Create a new entity
CREATE OR REPLACE TABLE link_all (
	-- Add a unique identifier to the link
	link_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
    -- Add history tracking attributes
	load_date TIMESTAMP,
    record_source VARCHAR(255)
);

In [ ]:
CREATE OR REPLACE TABLE link_all (
	link_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
    -- Add the hub's key attribute
	hub_employee_key NUMBER(10,0) UNIQUE,
  	load_date TIMESTAMP,
    record_source VARCHAR(255),
  	-- Add a relationship with the foreign hub
	FOREIGN KEY (hub_employee_key) REFERENCES hub_employee(hub_employee_key)
);

In [ ]:
CREATE OR REPLACE TABLE link_all (
	link_key NUMBER(10,0) AUTOINCREMENT PRIMARY KEY,
	hub_employee_key NUMBER(10,0) UNIQUE,
  	-- Add the hub's key attributes
  	hub_training_key NUMBER(10,0),
    hub_department_key NUMBER(10,0),
  	load_date TIMESTAMP,
    record_source VARCHAR(255),
	FOREIGN KEY (hub_employee_key) REFERENCES hub_employee(hub_employee_key),
  	-- Add a relationship with the foreign hubs
  	FOREIGN KEY (hub_training_key) REFERENCES hub_training(hub_training_key),
  	FOREIGN KEY (hub_department_key) REFERENCES hub_department( hub_department_key)
);

## Choosing the Right Approach

In [ ]:
SELECT   
hs.student_key,  
ss.student_name,
COUNT(le.class_key) AS NumberOfEnrollments 
MAX(sc.load_date) AS MostRecentEnrollmentDate 
FROM hub_students hs
JOIN sat_student ss ON hs.student_key = ss.student_key 
LEFT JOIN link_enrollment le ON hs.student_key = le.student_key 
LEFT JOIN sat_class sc ON le.class_key = sc.class_key 
GROUP BY hs.student_key, ss.student_name;

### Mastering data retrieval
After exploring various data modeling techniques, it's clear that the data vault model is ideally suited for this specific HR team's needs. The team requires a robust system that maintains historical records and tracks every change, a strength of the data vault model.

This exercise focuses on querying the data vault structure to analyze employees' training accomplishments and departmental affiliations.

Your task will be to gather the highest score for all employees that have won one award.

In [ ]:
SELECT
	-- Add the attribute from employees
    hub_e.hub_employee_key,
    -- Add the attribute from the department
    sat_d.department_name
FROM hub_employee AS hub_e
	-- Merge hub with link
	JOIN link_all AS li
    ON hub_e.hub_employee_key = li.hub_employee_key
	-- Merge link with satellite
    JOIN sat_department AS sat_d
    ON li.hub_department_key = sat_d.hub_department_key;

In [ ]:
SELECT
    hub_e.hub_employee_key,
    sat_d.department_name,
    -- Add the new attribute
	sat_t.avg_training_score
FROM hub_employee AS hub_e
	JOIN link_all AS li 
    ON hub_e.hub_employee_key = li.hub_employee_key
    JOIN sat_department AS sat_d 
    ON li.hub_department_key = sat_d.hub_department_key
    -- Merge the satellite, even if there is no data for that employee
    LEFT JOIN sat_training AS sat_t
    ON li.hub_training_key = sat_t.hub_training_key;

In [ ]:
SELECT
    hub_e.hub_employee_key,
    sat_d.department_name,
    sat_t.avg_training_score
FROM hub_employee AS hub_e
	JOIN link_all AS li 
    ON hub_e.hub_employee_key = li.hub_employee_key
    JOIN sat_department AS sat_d 
    ON li.hub_department_key = sat_d.hub_department_key
    LEFT JOIN sat_training AS sat_t 
    ON li.hub_training_key = sat_t.hub_training_key
-- Add filter for training
WHERE sat_t.awards_won = 1;

In [ ]:
SELECT
    hub_e.hub_employee_key,
    sat_d.department_name,
    -- Aggregate the attribute
    MAX(sat_t.avg_training_score) AS average_training
FROM hub_employee hub_e
	JOIN link_all AS li 
    ON hub_e.hub_employee_key = li.hub_employee_key
    JOIN sat_department AS sat_d 
    ON li.hub_department_key = sat_d.hub_department_key
    LEFT JOIN sat_training AS sat_t 
    ON li.hub_training_key = sat_t.hub_training_key
WHERE sat_t.awards_won = 1
-- Group the results 
GROUP BY hub_e.hub_employee_key, sat_d.department_name;